In [1]:
import sys, re, math, time
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import collections
from collections import OrderedDict
from matplotlib.pyplot import cm
%load_ext autoreload
%autoreload 2
from run_experiments import *
from datahelper import *
import keras.metrics
keras.metrics.cindex_score = cindex_score
from keras.models import load_model 
fpath = '../data/davis/'
%matplotlib inline

Using TensorFlow backend.


In [2]:
ligands = json.load(open(fpath+"ligands_can.txt"), object_pairs_hook=OrderedDict)
proteins = json.load(open(fpath+"proteins.txt"), object_pairs_hook=OrderedDict)

Y = pickle.load(open(fpath + "Y","rb"), encoding='latin1') ### TODO: read from raw

In [3]:
perfmeasure = get_cindex
runmethod = build_combined_categorical
FLAGS = argparser()
FLAGS.log_dir = FLAGS.log_dir + str(time.time()) + "/"
FLAGS.num_windows = 32
FLAGS.seq_window_lengths = [4]
FLAGS.smi_window_lengths = [4]
FLAGS.batch_size = 512
FLAGS.num_epoch = 1000
FLAGS.max_seq_len = 1000
FLAGS.max_smi_len = 100
FLAGS.dataset_path= '../data/davis/'
FLAGS.problem_type = 1
FLAGS.log_dir = '../logs'


In [4]:
dataset = DataSet( fpath = FLAGS.dataset_path, ### BUNU ARGS DA GUNCELLE
                      setting_no = FLAGS.problem_type, ##BUNU ARGS A EKLE
                      seqlen = FLAGS.max_seq_len,
                      smilen = FLAGS.max_smi_len,
                      need_shuffle = False )
# set character set size
FLAGS.charseqset_size = dataset.charseqset_size 
FLAGS.charsmiset_size = dataset.charsmiset_size 

XD, XT, Y = dataset.parse_data(fpath = FLAGS.dataset_path)

XD = np.asarray(XD)
XT = np.asarray(XT)
Y = np.asarray(Y)

drugcount = XD.shape[0]
print(drugcount)
targetcount = XT.shape[0]
print(targetcount)

FLAGS.drug_count = drugcount
FLAGS.target_count = targetcount

label_row_inds, label_col_inds = np.where(np.isnan(Y)==False)  #basically finds the point address of affinity [x,y]

if not os.path.exists(figdir):
    os.makedirs(figdir)
test_set, outer_train_sets = dataset.read_sets(FLAGS.dataset_path, FLAGS.problem_type) 


Reading ../data/davis/ start
Read ../data/davis/ start
68
442
Reading ../data/davis/ start


In [5]:

logging("---Parameter Search-----", FLAGS)

Y = np.mat(np.copy(Y))

params = {}

XD = XD[label_row_inds]
XT = XT[label_col_inds]

train_drugs, train_prots,  train_Y = prepare_interaction_pairs(XD, XT, Y, label_row_inds, label_col_inds)

In [6]:

data_path='../../data/'
XD_dtc, XT_dtc, Y_dtc = get_DTC_train(data_path+'dtc_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len)


In [7]:
from sklearn.model_selection import train_test_split
all_train_drugs = np.concatenate((np.asarray(train_drugs), np.asarray(XD_dtc)), axis=0)
all_train_prots = np.concatenate((np.asarray(train_prots), np.asarray(XT_dtc)), axis=0)
all_train_Y = np.concatenate((np.asarray(train_Y), np.asarray(Y_dtc)), axis=0)
all_train_Y = -np.log10(1e-8+all_train_Y/1e9)

unique_drugs = np.unique(all_train_drugs, axis=0)
np.random.seed(15)
choose_70_drugs = np.random.choice(np.arange(unique_drugs.shape[0]), 70, replace=False)
val_inds = []
for drug_ind in choose_70_drugs:
    val_inds += list(np.where((~(all_train_drugs==unique_drugs[drug_ind, :])).sum(axis=1) == 0)[0])

train_inds = np.delete(np.arange(all_train_drugs.shape[0]), val_inds)

    
XD_train, XD_val = all_train_drugs[train_inds, :], all_train_drugs[val_inds, :]
XT_train, XT_val = all_train_prots[train_inds, :], all_train_prots[val_inds, :]
Y_train, Y_val = all_train_Y[train_inds], all_train_Y[val_inds]

#XD_train, XD_val, XT_train, XT_val, Y_train, Y_val = train_test_split(all_train_drugs, all_train_prots, all_train_Y, 
#                                                                      test_size=0.1, 
#                                                                     random_state=15)


In [8]:
model_name='checkpoints/davis_dtc_dta'
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=25)
checkpoint_callback = ModelCheckpoint(model_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

gridmodel = build_combined_categorical(FLAGS, FLAGS.num_windows, FLAGS.smi_window_lengths[0], FLAGS.seq_window_lengths[0])
gridres = gridmodel.fit(([XD_train, XT_train ]), Y_train, batch_size=FLAGS.batch_size, epochs=FLAGS.num_epoch, 
        validation_data=( ([np.array(XD_val), np.array(XT_val) ]), np.array(Y_val))
           , callbacks=[early_stopping_callback, checkpoint_callback], verbose=2)

gridmodel.save('davis_dtc_dta.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 53955 samples, validate on 3151 samples
Epoch 1/1000
Epoch 00001: val_loss improved from inf to 1.87669, saving model to checkpoints/davis_dtc_dta.h5
 - 38s - loss: 1.3416 - cindex_score: 0.6434 - val_loss: 1.8767 - val_cindex_score: 0.3828
Epoch 2/1000
Epoch 00002: val_loss improved from 1.87669 to 1.38942, saving model to checkpoints/davis_dtc_dta.h5
 - 32s - loss: 0.5921 - cindex_score: 0.6925 - val_loss: 1.3894 - val_cindex_score: 0.3797
Epoch 3/1000
Epoch 00003: val_loss did not improve
 - 32s - loss: 0.5335 - cindex_score: 0.7202 - val_loss: 2.1693 - val_cindex_score: 0.5163
Epoch 4/1000
Epoch 00004: val_loss did not improve
 - 32s - loss: 0.5156 - cindex_score: 0.7262 - val_loss: 2.4572 - val_cindex_score: 0.6314
Epoch 5/1000
Epoch 00005: val_loss improved from 1.38942 to 1.21198, saving model to checkpoints/davis_dtc_dta.h5
 

In [11]:
from sklearn.metrics import mean_squared_error, auc, f1_score
from scipy.stats import pearsonr, spearmanr

gridmodel = load_model('checkpoints/davis_dtc_dta.h5')
predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val) ])
loss, rperf2 = gridmodel.evaluate(([np.array(XD_val),np.array(XT_val) ]), np.array(Y_val), verbose=0)


In [12]:
from sklearn.metrics import mean_squared_error, f1_score
predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val) ])
print('cindex:', get_cindex(Y_val, predicted_labels))
print('rmse:', np.sqrt(mean_squared_error(Y_val, predicted_labels)))
#print('pearsonr:', pearsonr(Y_val, predicted_labels[:, 0]))
#print('spearmanr:', np.sqrt(spearmanr(val_Y, predicted_labels[:, 0])))
print('f1:', np.sqrt(f1_score(Y_val>7, predicted_labels>7)))

#v0.1
#cindex: [0.81675458]
#rmse: 0.8683109279804
#f1: 0.6599120175960898
#checkpoint
#cindex: [0.79595819]
#rmse: 0.7753355061972348
#f1: 0.6260990336999411

#v0.2
#cindex: [0.82653753]
#rmse: 0.7971002447060451
#f1: 0.6114717550558165
#checkpoint
#cindex: [0.82653753]
#rmse: 0.7971002447060451
#f1: 0.6114717550558165

cindex: [0.82653753]
rmse: 0.7971002447060451
f1: 0.6114717550558165


In [ ]:
XD_test, XT_test = get_DTC_train(data_path+'dtc_test_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len, with_label=False)
XD_test, XT_test = np.asarray(XD_test), np.asarray(XT_test)
predicted_labels = gridmodel.predict([np.array(XD_test), np.array(XT_test) ])


In [ ]:
submission_data = pd.read_csv(data_path+'round_1_template.csv')
submission_data.loc[:, 'pKd_[M]_pred'] = predicted_labels
submission_data.to_csv(data_path+'submission_file.csv', index=None)